#Libraries n download file

In [1]:
!pip install category_encoders
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 840.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 7.3 MB/s eta 0:00:00


In [2]:
import category_encoders as ce
import math
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from catboost import CatBoostClassifier
from plotly.subplots import make_subplots
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.ensemble import VotingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, make_scorer
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from sklearn.svm import SVC
import lightgbm as lgb

In [3]:
!gdown 1c_ycZ6uHoOD1LR0NdZxkvCy-AhOf1KAp
!gdown 1ZtixaEVniVlITm0_B3WRxZQK5zmJqB-k
!gdown 1XrhyfQg_FC-jNBddxEkNDjovyD68JSeI
!gdown 1IK2sFj1PyTVzP35JiuFwYNBfum7DIG7n

Downloading...
From: https://drive.google.com/uc?id=1c_ycZ6uHoOD1LR0NdZxkvCy-AhOf1KAp
To: /content/delay_4_6_CONDITION_PRODUCT_SUPPLIER.csv
100% 2.09M/2.09M [00:00<00:00, 57.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ZtixaEVniVlITm0_B3WRxZQK5zmJqB-k
To: /content/not_delay_4_6_CONDITION_PRODUCT_SUPPLIER.csv
100% 83.4M/83.4M [00:03<00:00, 26.2MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1XrhyfQg_FC-jNBddxEkNDjovyD68JSeI
From (redirected): https://drive.google.com/uc?id=1XrhyfQg_FC-jNBddxEkNDjovyD68JSeI&confirm=t&uuid=898b0528-d185-459e-9972-fb6617221cb3
To: /content/not_delay_7_9_CONDITION_PRODUCT_SUPPLIER.csv
100% 181M/181M [00:02<00:00, 84.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1IK2sFj1PyTVzP35JiuFwYNBfum7DIG7n
To: /content/delay_7_9_CONDITION_PRODUCT_SUPPLIER.csv
100% 4.62M/4.62M [00:00<00:00, 51.5MB/s]


##Load dataframes

In [82]:
df_46delay = pd.read_csv('/content/delay_4_6_CONDITION_PRODUCT_SUPPLIER.csv')
df_46ndelay = pd.read_csv('/content/not_delay_4_6_CONDITION_PRODUCT_SUPPLIER.csv')
df_79delay = pd.read_csv('/content/delay_7_9_CONDITION_PRODUCT_SUPPLIER.csv')
df_79ndelay = pd.read_csv('/content/not_delay_7_9_CONDITION_PRODUCT_SUPPLIER.csv')

<ipython-input-82-f4bdf3c25f21>:2: DtypeWarning:

Columns (10,42) have mixed types. Specify dtype option on import or set low_memory=False.

<ipython-input-82-f4bdf3c25f21>:4: DtypeWarning:

Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.



#EDA

##Function def

###Plot <20 values features

In [48]:
def check_value_col(df):
    col = df.columns
    check_value_col = []
    for x in col:
        if(len(df[x].unique())<=20):
            check_value_col.append(x)
            print(x)
    print(f"we have {len(check_value_col)} columns have less than 20 unique values")
    return check_value_col
def plot_values_check(df):
    value_cols = check_value_col(df)
    num_row = math.ceil(len(value_cols)/3)
    value_c = []
    for x in value_cols:
        value_c.append(df[x].value_counts())

    subplot_titles = value_cols
    specs = [[{'type':'domain'}] * 3] * num_row
    fig = make_subplots(
            rows=num_row,
            cols=3,
            subplot_titles=subplot_titles,
            specs=specs,
            print_grid=True)
    count=0
    x = 0
    y = 0
    for value in value_c:
        x = count%3 +1
        y = count//3 + 1
        # this is the dataframe for every continent
        fig.add_trace(
            go.Pie(labels=value.index,
                values=value.values,
                showlegend=False,
                textposition='inside',
                textinfo='label+percent',
                hole =.3),
            row=y,
            col=x
        )
        count+=1;

    fig.update_layout(title="Percentage", title_x=0.5, width = 1000, height = 430*num_row)
    fig.show()

##4-6 delay

###Info

In [49]:
df_46delay.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9733 entries, 0 to 9732
Data columns (total 47 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Order date                       9733 non-null   object 
 1   SUBSIDIARY_CD                    9733 non-null   object 
 2   GLOBAL_NO                        9733 non-null   int64  
 3   CLASSIFY_CD                      9733 non-null   int64  
 4   CUST_CD                          9733 non-null   int64  
 5   BRAND_CD                         9733 non-null   object 
 6   INNER_CD                         9733 non-null   object 
 7   SUPPLIER_CD                      9733 non-null   object 
 8   Sales order line number          9733 non-null   int64  
 9   Stock class                      9733 non-null   int64  
 10  Consider count hodiday Saturday  9733 non-null   int64  
 11  SO QTY                           9733 non-null   int64  
 12  OTHER AREA SHIP DIV 

###Visualize

In [50]:
plot_values_check(df_46delay)

SUBSIDIARY_CD
Stock class
Consider count hodiday Saturday
OTHER AREA SHIP DIV
PACKING RANK
PRODUCT ATTRIBUTION
SPECIAL DIV
LOGICAL PLANT
DIRECT SHIP FLG
DELI_DIV
label
Ship Mode
SUPPLIER_DIV
HEAVY_FLG
EXPENSIVE_FLG
SPECIAL_DIV
SPECIFY_PRODUCTION_DAYS
HAZARD_FLG
SO_DAY_OF_WEEK
IO_UNFIT_FLG
WEIGHT_UNIT
SUPPLIER_CATEGORY_CD
PRODUCT_ASSORT
we have 23 columns have less than 20 unique values
This is the format of your plot grid:
[ (1,1)  ]  [ (1,2)  ]  [ (1,3)  ]
[ (2,1)  ]  [ (2,2)  ]  [ (2,3)  ]
[ (3,1)  ]  [ (3,2)  ]  [ (3,3)  ]
[ (4,1)  ]  [ (4,2)  ]  [ (4,3)  ]
[ (5,1)  ]  [ (5,2)  ]  [ (5,3)  ]
[ (6,1)  ]  [ (6,2)  ]  [ (6,3)  ]
[ (7,1)  ]  [ (7,2)  ]  [ (7,3)  ]
[ (8,1)  ]  [ (8,2)  ]  [ (8,3)  ]



## 4-6 notdelay

###Info

In [51]:
df_46ndelay.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 389320 entries, 0 to 389319
Data columns (total 47 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   Order date                       389320 non-null  object 
 1   SUBSIDIARY_CD                    389320 non-null  object 
 2   GLOBAL_NO                        389320 non-null  int64  
 3   CLASSIFY_CD                      389320 non-null  int64  
 4   CUST_CD                          389320 non-null  int64  
 5   BRAND_CD                         389320 non-null  object 
 6   INNER_CD                         389320 non-null  object 
 7   SUPPLIER_CD                      389320 non-null  object 
 8   Sales order line number          389320 non-null  int64  
 9   Stock class                      389320 non-null  int64  
 10  Consider count hodiday Saturday  389320 non-null  object 
 11  SO QTY                           389320 non-null  int64  
 12  OT

###Visualize

In [52]:
plot_values_check(df_46ndelay)

SUBSIDIARY_CD
Stock class
Consider count hodiday Saturday
OTHER AREA SHIP DIV
PACKING RANK
PRODUCT ATTRIBUTION
SPECIAL DIV
LOGICAL PLANT
DIRECT SHIP FLG
DELI_DIV
label
SUPPLIER_DIV
HEAVY_FLG
EXPENSIVE_FLG
SPECIAL_DIV
HAZARD_FLG
SO_DAY_OF_WEEK
IO_UNFIT_FLG
WEIGHT_UNIT
SUPPLIER_CATEGORY_CD
PRODUCT_ASSORT
we have 21 columns have less than 20 unique values
This is the format of your plot grid:
[ (1,1)  ]  [ (1,2)  ]  [ (1,3)  ]
[ (2,1)  ]  [ (2,2)  ]  [ (2,3)  ]
[ (3,1)  ]  [ (3,2)  ]  [ (3,3)  ]
[ (4,1)  ]  [ (4,2)  ]  [ (4,3)  ]
[ (5,1)  ]  [ (5,2)  ]  [ (5,3)  ]
[ (6,1)  ]  [ (6,2)  ]  [ (6,3)  ]
[ (7,1)  ]  [ (7,2)  ]  [ (7,3)  ]



## 7-9 delay

###Info

In [53]:
df_79delay.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26217 entries, 0 to 26216
Data columns (total 37 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Order date                       26217 non-null  object 
 1   SUBSIDIARY_CD                    26217 non-null  object 
 2   GLOBAL_NO                        26217 non-null  object 
 3   CLASSIFY_CD                      26217 non-null  int64  
 4   CUST_CD                          26217 non-null  int64  
 5   BRAND_CD                         26217 non-null  object 
 6   INNER_CD                         26217 non-null  object 
 7   SUPPLIER_CD                      26217 non-null  object 
 8   Sales order line number          26217 non-null  int64  
 9   Stock class                      26217 non-null  int64  
 10  Consider count hodiday Saturday  26217 non-null  int64  
 11  SO QTY                           26217 non-null  int64  
 12  OTHER AREA SHIP DI

###Visualize

In [54]:
plot_values_check(df_79delay)

SUBSIDIARY_CD
Stock class
Consider count hodiday Saturday
OTHER AREA SHIP DIV
PACKING RANK
PRODUCT ATTRIBUTION
SPECIAL DIV
LOGICAL PLANT
DIRECT SHIP FLG
DELI_DIV
label
Ship Mode
SUPPLIER_DIV
SPECIAL_DIV
SO_DAY_OF_WEEK
we have 15 columns have less than 20 unique values
This is the format of your plot grid:
[ (1,1)  ]  [ (1,2)  ]  [ (1,3)  ]
[ (2,1)  ]  [ (2,2)  ]  [ (2,3)  ]
[ (3,1)  ]  [ (3,2)  ]  [ (3,3)  ]
[ (4,1)  ]  [ (4,2)  ]  [ (4,3)  ]
[ (5,1)  ]  [ (5,2)  ]  [ (5,3)  ]



## 7-9 notdelay

###Info

In [55]:
df_79ndelay.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048680 entries, 0 to 1048679
Data columns (total 37 columns):
 #   Column                           Non-Null Count    Dtype  
---  ------                           --------------    -----  
 0   Order date                       1048680 non-null  object 
 1   SUBSIDIARY_CD                    1048680 non-null  object 
 2   GLOBAL_NO                        1048680 non-null  object 
 3   CLASSIFY_CD                      1048680 non-null  int64  
 4   CUST_CD                          1048680 non-null  int64  
 5   BRAND_CD                         1048680 non-null  object 
 6   INNER_CD                         1048680 non-null  object 
 7   SUPPLIER_CD                      1048680 non-null  object 
 8   Sales order line number          1048680 non-null  int64  
 9   Stock class                      1048680 non-null  int64  
 10  Consider count hodiday Saturday  1048680 non-null  int64  
 11  SO QTY                           1048680 non-null 

###Visualize

In [56]:
plot_values_check(df_79ndelay)

SUBSIDIARY_CD
Stock class
Consider count hodiday Saturday
OTHER AREA SHIP DIV
PACKING RANK
PRODUCT ATTRIBUTION
SPECIAL DIV
LOGICAL PLANT
DIRECT SHIP FLG
DELI_DIV
label
SUPPLIER_DIV
SPECIAL_DIV
SO_DAY_OF_WEEK
we have 14 columns have less than 20 unique values
This is the format of your plot grid:
[ (1,1)  ]  [ (1,2)  ]  [ (1,3)  ]
[ (2,1)  ]  [ (2,2)  ]  [ (2,3)  ]
[ (3,1)  ]  [ (3,2)  ]  [ (3,3)  ]
[ (4,1)  ]  [ (4,2)  ]  [ (4,3)  ]
[ (5,1)  ]  [ (5,2)  ]  [ (5,3)  ]



#FEATURES ENGINEERING

##CONVERT DATA TYPES

In [83]:
def convert_dtypes(df):
    #Date
    date_type_col = ['Order date', 'VSD']
    for x in date_type_col:
        df[x] = pd.to_datetime(df[x], format = 'mixed')

    #FillNA
    df["Ship Mode"] = df["Ship Mode"].fillna(1)
    df["OTHER AREA SHIP DIV"] = df["OTHER AREA SHIP DIV"].fillna(0)

    #Int
    df["OTHER AREA SHIP DIV"] = df["OTHER AREA SHIP DIV"].replace([" "], 0)
    df['Consider count hodiday Saturday'] = df['Consider count hodiday Saturday'].replace([" "], 0)
    df['Consider count hodiday Saturday'] = df['Consider count hodiday Saturday'].astype(int)
    df['Sales order line number'] = df['Sales order line number'].astype(int)
    df['OTHER AREA SHIP DIV'] = df['OTHER AREA SHIP DIV'].astype(int)
    df['Consider count hodiday Saturday'] = df['Consider count hodiday Saturday'].astype(int)
    df['Sales order line number'] = df['Sales order line number'].astype(int)

    return df

In [84]:
df_46delay = convert_dtypes(df_46delay)
df_46ndelay = convert_dtypes(df_46ndelay)
df_79delay = convert_dtypes(df_79delay)
df_79ndelay = convert_dtypes(df_79ndelay)

##DROP FEATURES

Using above visualized result and previous lab result to drop features

Maybe, You ask me why I dropped those poor little features. It's "kinh nghiệm và trải nghiệm" <(")

In [85]:
drop_col = ['SPECIAL_DIV', 'PRODUCT ATTRIBUTION', 'SUBSIDIARY_CD','Stock class', 'PACK QTY','GLOBAL_NO','SHIP DECISION NO','SUPPLIER_DIV','REASON_CD','PURCHASE AMOUNT']
df_46delay = df_46delay.drop(columns = drop_col)
df_46ndelay = df_46ndelay.drop(columns = drop_col)
df_79delay = df_79delay.drop(columns = drop_col)
df_79ndelay = df_79ndelay.drop(columns = drop_col)

In [86]:
no_recorded = ['ACTUAL_SHIP_DAYS','SPECIFY_PRODUCTION_DAYS','SPECIFY_SHIP_DAYS','PRODUCT_ASSORT','WEIGHT_UNIT','HAZARD_FLG','IO_UNFIT_FLG','EXPENSIVE_FLG','HEAVY_FLG','SUPPLIER_CATEGORY_CD']
df_46delay = df_46delay.drop(columns = no_recorded)
df_46ndelay = df_46ndelay.drop(columns = no_recorded)

In [87]:
def drop_null_features(df, show_chart, percent):
    # df: Data frame
    # show_chart [True, False] : present ratio chart
    # percent [0,1] : Percentage for dropping feature which have more "null" percent than that
    ratios = []
    mostly_null_record = []
    for feature in df.columns:
        num_total_records = df[feature].shape[0]
        value_counts = df[feature].isnull().value_counts()
        num_not_null_records = value_counts[False]
        ratio = (num_total_records - num_not_null_records) / num_total_records
        if(ratio >= percent):
            mostly_null_record.append(feature)
        ratios.append(ratio)
    if(show_chart == True):
        fig = go.Figure([go.Bar(x=df.columns, y= ratios)])
        fig.update_xaxes(tickangle=45)
        fig.show()
    df = df.drop(columns = mostly_null_record)
    print(f"Dropped {mostly_null_record} for having more than {percent*100}% null")
    return df

In [88]:
df_46delay = drop_null_features(df_46delay, True, 0.90)

Dropped ['SOUF_RCV_NO', 'QTUF_RCV_NO'] for having more than 90.0% null


In [89]:
df_46ndelay = drop_null_features(df_46ndelay, True, 0.90)

Dropped ['SOUF_RCV_NO', 'QTUF_RCV_NO'] for having more than 90.0% null


In [90]:
df_79delay = drop_null_features(df_79delay, True, 0.90)

Dropped ['SOUF_RCV_NO', 'QTUF_RCV_NO'] for having more than 90.0% null


In [91]:
df_79ndelay = drop_null_features(df_79ndelay, True, 0.9)

Dropped ['SOUF_RCV_NO', 'QTUF_RCV_NO'] for having more than 90.0% null


##MERGE FEATURES

In [92]:
def cal_weight(df):
    WEIGHT = []
    for i in range(len(df)):
        w = df['SO QTY'].values[i] * df['WEIGHT PER PIECE'].values[i]
        WEIGHT.append(w)
    df['WEIGHT'] = WEIGHT
    weight_record = ['SO QTY','WEIGHT PER PIECE']
    df = df.drop(columns = weight_record)
    print("calculated!")
    return df

def cal_delivery_time(df):
    DELIVERY_TIME = []
    for i in range(len(df)):
        d = df['VSD'].values[i] - df['Order date'].values[i]
        DELIVERY_TIME.append(d)

    df['DELIVERY_TIME'] = DELIVERY_TIME
    df['DELIVERY_TIME'] = df['DELIVERY_TIME'].dt.days
    time_record = ['VSD','Order date']
    df = df.drop(columns = time_record)
    print("calculated!")
    return df

def cal_all(df):
    df = cal_delivery_time(df)
    df = cal_weight(df)
    return df

In [93]:
df_46delay = cal_all(df_46delay)
df_46ndelay = cal_all(df_46ndelay)
df_79delay = cal_all(df_79delay)
df_79ndelay = cal_all(df_79ndelay)

calculated!
calculated!
calculated!
calculated!
calculated!
calculated!
calculated!
calculated!


## REMOVE OUTLINER

In [94]:
df_79ndelay.describe()

,CLASSIFY_CD,CUST_CD,Sales order line number,Consider count hodiday Saturday,OTHER AREA SHIP DIV,ALLOCATION QTY,SUPPLIER INV AMOUNT,SPECIAL DIV,LOGICAL PLANT,DIRECT SHIP FLG,label,SO_DAY_OF_MONTH,SO_DAY_OF_WEEK,SO_TIME,DELIVERY_TIME,WEIGHT
count,1.048680e+06,1.048680e+06,1.048680e+06,1.048680e+06,1.048680e+06,1.048680e+06,1.048680e+06,1.048680e+06,1.048680e+06,1.048680e+06,1048680.0,1.048680e+06,1.048680e+06,1.048680e+06,1.048680e+06,1.048680e+06
mean,5.977979e+06,2.438826e+05,1.518312e+02,1.137931e+00,1.493735e-01,7.592469e+00,5.596148e+03,2.371934e-02,5.337561e+01,9.384846e-02,0.0,1.538362e+01,3.157605e+00,1.344735e+05,1.817149e+01,1.998494e+03
std,6.330669e+06,2.461097e+05,3.326032e+03,1.781909e+00,3.564564e-01,1.314869e+02,2.841218e+04,1.521734e-01,1.156626e+00,2.916179e-01,0.0,9.139567e+00,1.463858e+00,3.219387e+04,3.701671e+01,1.497058e+04
min,1.030101e+06,1.000000e+01,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,8.400000e-01,0.000000e+00,5.200000e+01,0.000000e+00,0.0,1.000000e+00,1.000000e+00,1.040700e+04,0.000000e+00,0.000000e+00
25%,3.633521e+06,6.146500e+04,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,7.030000e+02,0.000000e+00,5.300000e+01,0.000000e+00,0.0,7.000000e+00,2.000000e+00,1.053030e+05,3.000000e+00,4.600000e+01
50%,3.733118e+06,1.320230e+05,3.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,1.748000e+03,0.000000e+00,5.300000e+01,0.000000e+00,0.0,1.500000e+01,3.000000e+00,1.357510e+05,6.000000e+00,2.200000e+02
75%,3.954110e+06,4.058510e+05,1.200000e+01,4.000000e+00,0.000000e+00,4.000000e+00,4.494000e+03,0.000000e+00,5.500000e+01,0.000000e+00,0.0,2.300000e+01,4.000000e+00,1.621210e+05,1.500000e+01,1.021000e+03
max,9.901000e+07,9.999130e+05,9.005000e+04,4.000000e+00,1.000000e+00,9.000000e+04,1.214400e+07,1.000000e+00,5.500000e+01,1.000000e+00,0.0,3.100000e+01,6.000000e+00,2.030000e+05,4.150000e+02,4.494900e+06


In [95]:
df = df_79ndelay['SUPPLIER INV AMOUNT'].value_counts().sort_index()
pd.set_option('display.max_rows', None)
print(df)

SUPPLIER INV AMOUNT
0.84              2
0.95              1
0.96              1
1.00            244
1.01             60
1.03             20
1.05             43
1.06              1
1.07             35
1.08              6
1.10             34
1.12              1
1.13              9
1.14             25
1.15              4
1.16             33
1.17              3
1.18             11
1.19             20
1.20             37
1.21             19
1.23             10
1.25              9
1.26              2
1.27              7
1.28             10
1.29             21
1.30              1
1.33             38
1.34              6
1.35              7
1.36             16
1.37             32
1.38             11
1.39              6
1.40              2
1.41              1
1.42              8
1.43              4
1.44              2
1.45              2
1.46              8
1.48              3
1.49              3
1.50             15
1.51              8
1.52              3
1.53              1
1.54              5


In [96]:
outlier_cols = ['Sales order line number','ALLOCATION QTY','SUPPLIER INV AMOUNT']

In [97]:
def remove_outlier(df, cols):
    for col_name  in cols:
        q1 = df[col_name].quantile(0.25)
        q3 = df[col_name].quantile(0.75)
        iqr = q3 - q1
        fence_low = q1 - 1.5 * iqr
        fence_high = q3 + 1.5 * iqr
        df = df.loc[(df[col_name] > fence_low) & (df[col_name] < fence_high)]
    return df

In [98]:
df_46delay = remove_outlier(df_46delay, outlier_cols)
df_46ndelay = remove_outlier(df_46ndelay, outlier_cols)
df_79delay = remove_outlier(df_79delay, outlier_cols)
df_79ndelay = remove_outlier(df_79ndelay, outlier_cols)

In [99]:
df_79ndelay.describe()

,CLASSIFY_CD,CUST_CD,Sales order line number,Consider count hodiday Saturday,OTHER AREA SHIP DIV,ALLOCATION QTY,SUPPLIER INV AMOUNT,SPECIAL DIV,LOGICAL PLANT,DIRECT SHIP FLG,label,SO_DAY_OF_MONTH,SO_DAY_OF_WEEK,SO_TIME,DELIVERY_TIME,WEIGHT
count,7.235260e+05,723526.000000,723526.000000,723526.000000,723526.000000,723526.000000,723526.000000,723526.000000,723526.000000,723526.000000,723526.0,723526.000000,723526.000000,723526.000000,723526.000000,723526.000000
mean,5.773144e+06,248470.029964,5.628445,1.083675,0.146751,2.358078,2011.289986,0.010348,53.387435,0.085904,0.0,15.367229,3.150676,134038.226126,14.317593,858.751174
std,6.075395e+06,250618.499136,6.333920,1.754277,0.353858,1.740148,1897.343923,0.101197,1.166572,0.280223,0.0,9.154887,1.463800,31918.072023,31.408024,2955.197015
min,1.030101e+06,10.000000,1.000000,0.000000,0.000000,1.000000,0.840000,0.000000,52.000000,0.000000,0.0,1.000000,1.000000,10407.000000,0.000000,1.000000
25%,3.633521e+06,55533.000000,1.000000,0.000000,0.000000,1.000000,602.000000,0.000000,53.000000,0.000000,0.0,7.000000,2.000000,105151.000000,3.000000,39.000000
50%,3.733118e+06,132206.000000,3.000000,0.000000,0.000000,2.000000,1332.000000,0.000000,53.000000,0.000000,0.0,15.000000,3.000000,135424.000000,5.000000,160.000000
75%,3.954109e+06,410983.000000,8.000000,4.000000,0.000000,3.000000,2830.000000,0.000000,55.000000,0.000000,0.0,23.000000,4.000000,161754.000000,11.000000,720.000000
max,9.901000e+07,999913.000000,28.000000,4.000000,1.000000,8.000000,8549.000000,1.000000,55.000000,1.000000,0.0,31.000000,6.000000,203000.000000,399.000000,400000.000000


##ENCODE

I tried count encoding, ordinal encoding, I see target encoding is the best

In [100]:
def encode_oasd(df):
    # Replace "  " value to 1
    # Replace "1" value to 0
    df["OTHER AREA SHIP DIV"] = df["OTHER AREA SHIP DIV"].replace([" "], 1)
    df["OTHER AREA SHIP DIV"] = df["OTHER AREA SHIP DIV"].replace(["1"], 0)
    print(f"encode complete!")
    return df

def encode_categories(df, encode_type):
    cat_features = [c for c in df.columns if ( df[c].dtype not in [np.float64, np.int64])]
    df[cat_features] = df[cat_features].astype(str)
    if(encode_type == "count"):
        count_enc = ce.CountEncoder()
        encoded_features = count_enc.fit_transform(df[cat_features])
        df[cat_features] = encoded_features
        print(f"encode complete!")
        return df
    elif(encode_type == "ordinal"):
        ordinal_enc = OrdinalEncoder()
        encoded_features = ordinal_enc.fit_transform(df[cat_features])
        df[cat_features] = encoded_features
        print(f"encode complete!")
        return df
    elif(encode_type == "target"):
        target = df[['label']]
        train = df.drop('label',axis = 1)
        cbe_encoder = ce.cat_boost.CatBoostEncoder()
        train_cbe = cbe_encoder.fit_transform(train,target)
        train_cbe['label'] = target
        df = train_cbe
        return df
def encode_so_time(df):
    ENCODED_SO_TIME = []
    for i in range(len(df)):
        time = df['SO_TIME'].values[i]
        if (time > 30000 and time <=70000):
            e = 0
        elif (time > 70000 and time <=110000):
            e = 1
        elif (time > 110000 and time <=150000):
            e = 2
        elif (time > 150000 and time <=190000):
            e = 3
        elif (time > 190000 and time <=220000):
            e = 4
        else:
            e = 5
        ENCODED_SO_TIME.append(e)
    df['ENCODED_SO_TIME'] = ENCODED_SO_TIME
    df = df.drop(columns = ['SO_TIME'])
    print("encode complete!")
    return df
def encode_all(df):
    df = encode_oasd(df)
    df = encode_so_time(df)
    df = encode_categories(df, "target")
    return df

In [101]:
#For CatBoosting
df_ne46delay = df_46delay
df_ne46ndelay = df_46ndelay
df_ne79delay = df_79delay
df_ne79ndelay = df_79ndelay
cate_features = [c for c in df_ne46delay.columns if ( df_ne46delay[c].dtype not in [np.float64, np.int64])]

#For every homie!
df_46delay = encode_all(df_46delay)
df_46ndelay = encode_all(df_46ndelay)
df_79delay = encode_all(df_79delay)
df_79ndelay = encode_all(df_79ndelay)

encode complete!
encode complete!
encode complete!
encode complete!
encode complete!
encode complete!
encode complete!
encode complete!
encode complete!
encode complete!
encode complete!
encode complete!


## SCALE DATA

In [102]:
def scale_data(df):
    #When using target encoding, we dont need to use scaler
    # df['CLASSIFY_CD'] = stats.zscore(df['CLASSIFY_CD'])
    # df['CUST_CD'] = stats.zscore(df['CUST_CD'])
    # df['INNER_CD'] = stats.zscore(df['INNER_CD'])
    scaler = MinMaxScaler()
    df['SUPPLIER INV AMOUNT'] = scaler.fit_transform(df[['SUPPLIER INV AMOUNT']])
    df['WEIGHT'] = scaler.fit_transform(df[['WEIGHT']])
    # df['SUPPLIER_CD'] = scaler.fit_transform(df[['SUPPLIER_CD']])
    # df['PRODUCT_CD'] = scaler.fit_transform(df[['PRODUCT_CD']])
    return df

In [103]:
df_46delay = scale_data(df_46delay)
df_46ndelay = scale_data(df_46ndelay)
df_79delay = scale_data(df_79delay)
df_79ndelay = scale_data(df_79ndelay)

##SUMMARY

In [104]:
df_46delay.describe()

,CLASSIFY_CD,CUST_CD,BRAND_CD,INNER_CD,SUPPLIER_CD,Sales order line number,Consider count hodiday Saturday,OTHER AREA SHIP DIV,ALLOCATION QTY,SUPPLIER INV AMOUNT,...,LOGICAL PLANT,DIRECT SHIP FLG,DELI_DIV,label,Ship Mode,SO_DAY_OF_MONTH,SO_DAY_OF_WEEK,DELIVERY_TIME,WEIGHT,ENCODED_SO_TIME
count,6.505000e+03,6.505000e+03,6505.000000,6.505000e+03,6505.000000,6505.000000,6505.000000,6505.000000,6505.000000,6505.000000,...,6505.000000,6505.000000,6505.000000,6505.0,6505.000000,6505.000000,6505.000000,6505.000000,6505.000000,6505.000000
mean,6.772275e-17,-2.184605e-17,231.896541,1.135995e-16,0.290879,5.255035,1.367102,0.321599,2.837663,0.195129,...,53.354343,0.121752,0.325442,1.0,7.680092,16.824289,2.921599,11.985396,0.019410,2.196925
std,1.000077e+00,1.000077e+00,62.049589,1.000077e+00,0.259652,5.803489,1.875727,0.467126,2.385463,0.222200,...,1.046302,0.327025,0.887251,0.0,4.672110,7.250993,1.521172,13.187663,0.051967,0.847609
min,-9.218957e-01,-1.003475e+00,0.000000,-1.742010e+00,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,...,52.000000,0.000000,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000
25%,-5.436909e-01,-8.018175e-01,238.000000,-5.022729e-01,0.150485,1.000000,0.000000,0.000000,1.000000,0.040497,...,53.000000,0.000000,0.000000,1.0,2.000000,12.000000,1.000000,4.000000,0.001777,2.000000
50%,-5.436908e-01,-4.951067e-01,238.000000,-2.786598e-01,0.150485,3.000000,0.000000,0.000000,2.000000,0.104666,...,53.000000,0.000000,0.000000,1.0,10.000000,18.000000,3.000000,7.000000,0.006009,2.000000
75%,-4.036246e-01,8.965567e-01,238.000000,3.494300e-01,0.407767,7.000000,4.000000,1.000000,4.000000,0.257654,...,55.000000,0.000000,0.000000,1.0,12.000000,22.000000,4.000000,16.000000,0.017848,3.000000
max,2.018421e+00,2.733914e+00,453.000000,2.667772e+00,1.000000,25.000000,4.000000,1.000000,10.000000,1.000000,...,55.000000,1.000000,4.000000,1.0,17.000000,31.000000,6.000000,134.000000,1.000000,4.000000


In [105]:
df_46ndelay.describe()

,CLASSIFY_CD,CUST_CD,BRAND_CD,INNER_CD,SUPPLIER_CD,Sales order line number,Consider count hodiday Saturday,OTHER AREA SHIP DIV,ALLOCATION QTY,SUPPLIER INV AMOUNT,...,LOGICAL PLANT,DIRECT SHIP FLG,DELI_DIV,label,Ship Mode,SO_DAY_OF_MONTH,SO_DAY_OF_WEEK,DELIVERY_TIME,WEIGHT,ENCODED_SO_TIME
count,2.702000e+05,2.702000e+05,270200.000000,2.702000e+05,270200.000000,270200.000000,270200.000000,270200.000000,270200.000000,270200.000000,...,270200.000000,270200.000000,270200.000000,270200.0,270200.000000,270200.000000,270200.000000,270200.000000,270200.000000,270200.000000
mean,6.185035e-17,7.447287e-17,711.229027,-2.945255e-17,0.244923,5.574449,1.022417,0.155144,2.337657,0.237315,...,53.410470,0.091621,0.369604,0.0,10.047768,16.017506,3.058386,15.269759,0.002182,2.156129
std,1.000002e+00,1.000002e+00,121.320182,1.000002e+00,0.244196,6.358382,1.718965,0.362043,1.732756,0.221619,...,1.203473,0.288491,1.233455,0.0,4.695394,8.417218,1.475817,31.974507,0.007737,0.828472
min,-7.835740e-01,-9.754212e-01,0.000000,-1.697209e+00,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,...,52.000000,0.000000,0.000000,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000
25%,-3.541611e-01,-7.775988e-01,708.000000,-7.874765e-01,0.026549,1.000000,0.000000,0.000000,1.000000,0.072266,...,52.000000,0.000000,0.000000,0.0,4.000000,9.000000,2.000000,3.000000,0.000098,1.000000
50%,-3.377333e-01,-4.869042e-01,708.000000,1.078288e-01,0.222714,3.000000,0.000000,0.000000,2.000000,0.158216,...,53.000000,0.000000,0.000000,0.0,12.000000,16.000000,3.000000,5.000000,0.000418,2.000000
75%,-3.012827e-01,6.945115e-01,708.000000,6.556369e-01,0.269912,8.000000,1.000000,0.000000,3.000000,0.333384,...,55.000000,0.000000,0.000000,0.0,14.000000,23.000000,4.000000,12.000000,0.001873,3.000000
max,1.537741e+01,2.894144e+00,1324.000000,2.404581e+00,1.000000,28.000000,4.000000,1.000000,8.000000,1.000000,...,55.000000,1.000000,8.000000,0.0,18.000000,31.000000,6.000000,365.000000,1.000000,4.000000


In [106]:
df_79delay.describe()

,CLASSIFY_CD,CUST_CD,BRAND_CD,INNER_CD,SUPPLIER_CD,Sales order line number,Consider count hodiday Saturday,OTHER AREA SHIP DIV,ALLOCATION QTY,SUPPLIER INV AMOUNT,...,LOGICAL PLANT,DIRECT SHIP FLG,DELI_DIV,label,Ship Mode,SO_DAY_OF_MONTH,SO_DAY_OF_WEEK,DELIVERY_TIME,WEIGHT,ENCODED_SO_TIME
count,1.715500e+04,1.715500e+04,17155.000000,1.715500e+04,17155.000000,17155.000000,17155.000000,17155.000000,17155.000000,17155.000000,...,17155.000000,17155.000000,17155.000000,17155.0,17155.000000,17155.000000,17155.000000,17155.000000,17155.000000,17155.000000
mean,-7.455418e-18,-9.692043e-17,355.142233,7.517546e-17,0.349088,10.983737,1.501253,0.240746,3.048091,0.201383,...,53.206470,0.574352,1.703177,1.0,9.681026,14.129117,3.188167,15.525444,0.000967,2.214515
std,1.000029e+00,1.000029e+00,95.074851,1.000029e+00,0.254391,14.561510,1.901055,0.427549,2.685242,0.222893,...,1.148136,0.494455,1.468462,0.0,4.097581,8.709311,1.517006,21.329679,0.007857,0.844524
min,-9.258548e-01,-1.070669e+00,0.000000,-1.644230e+00,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,...,52.000000,0.000000,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000
25%,-5.793818e-01,-8.213062e-01,353.000000,-5.204241e-01,0.218254,1.000000,0.000000,0.000000,1.000000,0.043840,...,52.000000,0.000000,0.000000,1.0,8.000000,7.000000,2.000000,5.000000,0.000038,2.000000
50%,-5.517310e-01,-4.846079e-01,353.000000,-3.046958e-01,0.244048,4.000000,0.000000,0.000000,2.000000,0.112217,...,53.000000,1.000000,3.000000,1.0,10.000000,13.000000,3.000000,8.000000,0.000230,2.000000
75%,5.035928e-01,9.245194e-01,353.000000,4.963148e-01,0.509921,15.000000,4.000000,0.000000,4.000000,0.274426,...,55.000000,1.000000,3.000000,1.0,13.000000,21.000000,5.000000,18.000000,0.000919,3.000000
max,2.013120e+00,2.897440e+00,677.000000,2.473735e+00,1.000000,63.000000,4.000000,1.000000,13.000000,1.000000,...,55.000000,1.000000,5.000000,1.0,17.000000,31.000000,6.000000,369.000000,1.000000,4.000000


In [107]:
df_79ndelay.describe()

,CLASSIFY_CD,CUST_CD,BRAND_CD,INNER_CD,SUPPLIER_CD,Sales order line number,Consider count hodiday Saturday,OTHER AREA SHIP DIV,ALLOCATION QTY,SUPPLIER INV AMOUNT,...,LOGICAL PLANT,DIRECT SHIP FLG,DELI_DIV,label,Ship Mode,SO_DAY_OF_MONTH,SO_DAY_OF_WEEK,DELIVERY_TIME,WEIGHT,ENCODED_SO_TIME
count,7.235260e+05,7.235260e+05,723526.000000,7.235260e+05,723526.000000,723526.000000,723526.000000,723526.000000,723526.000000,723526.000000,...,723526.000000,723526.000000,723526.000000,723526.0,723526.000000,723526.000000,723526.000000,723526.000000,723526.000000,723526.000000
mean,2.035310e-17,3.201992e-17,854.719562,6.414787e-17,0.249830,5.628445,1.083675,0.146751,2.358078,0.235191,...,53.387435,0.085904,0.347367,0.0,10.814500,15.367229,3.150676,14.317593,0.002144,2.149620
std,1.000001e+00,1.000001e+00,141.086604,1.000001e+00,0.245206,6.333920,1.754277,0.353858,1.740148,0.221959,...,1.166572,0.280223,1.192140,0.0,4.875697,9.154887,1.463800,31.408024,0.007388,0.842312
min,-7.806976e-01,-9.913881e-01,0.000000,-1.624903e+00,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,...,52.000000,0.000000,0.000000,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000
25%,-3.521787e-01,-7.698441e-01,851.000000,-7.659043e-01,0.029650,1.000000,0.000000,0.000000,1.000000,0.070326,...,53.000000,0.000000,0.000000,0.0,6.000000,7.000000,2.000000,3.000000,0.000095,1.000000
50%,-3.357852e-01,-4.639087e-01,851.000000,8.267888e-02,0.225067,3.000000,0.000000,0.000000,2.000000,0.155725,...,53.000000,0.000000,0.000000,0.0,13.000000,15.000000,3.000000,5.000000,0.000398,2.000000
75%,-2.994104e-01,6.484481e-01,851.000000,5.749442e-01,0.277628,8.000000,4.000000,0.000000,3.000000,0.330967,...,55.000000,0.000000,0.000000,0.0,15.000000,23.000000,4.000000,11.000000,0.001798,3.000000
max,1.534664e+01,2.998356e+00,1571.000000,2.581503e+00,1.000000,28.000000,4.000000,1.000000,8.000000,1.000000,...,55.000000,1.000000,8.000000,0.0,20.000000,31.000000,6.000000,399.000000,1.000000,5.000000


#MODELING

##LABEL RATIO PARTITION ()

In [39]:
def label_ratio(dfA, dfB, pA, pB, num = 0):
    len_dfA = len(dfA)
    len_dfB = len(dfB)


    if(num == 0):
        num_dfB_records = min((len_dfA * pB) // pA, len_dfB)
        num_dfA_records = (pA * num_dfB_records) // pB
    else:
        part = round(num / (pA + pB))
        if(part*pB > len_dfB):
            part = round(len_dfB / pB)
        num_dfA_records = part*pA
        num_dfB_records = part*pB
    sampled_dfA = dfA.sample(n=num_dfA_records, random_state=1)
    sampled_dfB = dfB.sample(n=num_dfB_records, random_state=1)

    dfC = pd.concat([sampled_dfA, sampled_dfB]).reset_index(drop=True)

    print(f"New dataframe created with {len(dfC)} records: {num_dfA_records} from dfA and {num_dfB_records} from dfB.")
    return dfC.sample(frac=1, random_state=42)


Train-test set for CatBoosting

In [113]:
df_netrain = label_ratio(df_ne46ndelay, df_ne46delay, 3, 1, 15000)
X_netrain = df_netrain.drop(columns=['label'])
y_netrain = df_netrain['label']
df_netest = label_ratio(df_ne79ndelay, df_ne79delay, 3, 1, 3000)
X_netest = df_netest.drop(columns=['label'])
y_netest = df_netest['label']

New dataframe created with 15000 records: 11250 from dfA and 3750 from dfB.
New dataframe created with 3000 records: 2250 from dfA and 750 from dfB.


Train-test set for others

In [115]:
df_train = label_ratio(df_46ndelay, df_46delay, 3, 1,15000)
X_train = df_train.drop(columns=['label'])
y_train = df_train['label']
df_test = label_ratio(df_79ndelay, df_79delay, 3, 1, 3000)
X_test = df_test.drop(columns=['label'])
y_test = df_test['label']

New dataframe created with 15000 records: 11250 from dfA and 3750 from dfB.
New dataframe created with 3000 records: 2250 from dfA and 750 from dfB.


##MODEL

### Model#1: SVM

GRID SEARCH

In [117]:
def SVM_GS(X_train, y_train):
    model = SVC()
    param = {
        'C': [6, 7, 8],
        'tol': [0.001, 0.005, 0.01],
        'gamma': [0.0005, 0.001, 0.005],
        'kernel': ['rbf']
    }
    grid_search = GridSearchCV(estimator = model, param_grid = param, cv = 5, verbose= 10)

    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_

    # Fits the model to the training data.
    best_model.fit(X_train, y_train)
    print('The parameters combination that would give best accuracy is : ')
    print(grid_search.best_params_)
    print('The best accuracy achieved after parameter tuning via grid search is : ', grid_search.best_score_)
SVM_GS(X_train, y_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV 1/5; 1/27] START C=6, gamma=0.0005, kernel=rbf, tol=0.001...................
[CV 1/5; 1/27] END C=6, gamma=0.0005, kernel=rbf, tol=0.001;, score=0.989 total time=   0.6s
[CV 2/5; 1/27] START C=6, gamma=0.0005, kernel=rbf, tol=0.001...................
[CV 2/5; 1/27] END C=6, gamma=0.0005, kernel=rbf, tol=0.001;, score=0.988 total time=   0.6s
[CV 3/5; 1/27] START C=6, gamma=0.0005, kernel=rbf, tol=0.001...................
[CV 3/5; 1/27] END C=6, gamma=0.0005, kernel=rbf, tol=0.001;, score=0.985 total time=   0.5s
[CV 4/5; 1/27] START C=6, gamma=0.0005, kernel=rbf, tol=0.001...................
[CV 4/5; 1/27] END C=6, gamma=0.0005, kernel=rbf, tol=0.001;, score=0.992 total time=   0.6s
[CV 5/5; 1/27] START C=6, gamma=0.0005, kernel=rbf, tol=0.001...................
[CV 5/5; 1/27] END C=6, gamma=0.0005, kernel=rbf, tol=0.001;, score=0.989 total time=   0.5s
[CV 1/5; 2/27] START C=6, gamma=0.0005, kernel=rbf, tol=0.005.......

In [118]:
def SVM_model(X_train, y_train, X_test, y_test):
    model = SVC(kernel='rbf', C=8, gamma =0.001, tol =0.005)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    SVM_F1 = f1_score(y_test, y_pred)
    SVM_acc = accuracy_score(y_test, y_pred)
    SVM_cm = confusion_matrix(y_test, y_pred)
    print(f"F1 score: {SVM_F1}")
    print(f"Accuracy: {SVM_acc}")
    print(f"Confussion matrix{SVM_cm}")

In [119]:
SVM_model(X_train, y_train, X_test, y_test)

F1 score: 0.8687454677302392
Accuracy: 0.9396666666666667
Confussion matrix[[2220   30]
 [ 151  599]]




```
With 5:1
F1 score: 0.5485232067510548
Accuracy: 0.8928928928928929
Confussion matrix[[1654   11]
                  [ 203  130]]

With 3:1
F1 score: 0.8687454677302392
Accuracy: 0.9396666666666667
Confussion matrix[[2220   30]
                  [ 151  599]]
```



###Model#2: KNN

In [121]:
def KNN_GS(X_train, y_train):
    pca = PCA(n_components = 15, random_state=42)
    X_train = pca.fit_transform(X_train)
    model = KNeighborsClassifier()
    param = {
        'algorithm': ['ball_tree', 'kd_tree', 'brute'],
        'weights': ['uniform', 'distance'],
        'n_neighbors' : [6, 8, 10]
    }
    grid_search = GridSearchCV(estimator = model, param_grid = param, cv = 5, verbose= 10)

    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_

    # Fits the model to the training data.
    best_model.fit(X_train, y_train)
    print('The parameters combination that would give best accuracy is : ')
    print(grid_search.best_params_)
    print('The best accuracy achieved after parameter tuning via grid search is : ', grid_search.best_score_)
    return pca
pca = KNN_GS(X_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5; 1/18] START algorithm=ball_tree, n_neighbors=6, weights=uniform........
[CV 1/5; 1/18] END algorithm=ball_tree, n_neighbors=6, weights=uniform;, score=0.988 total time=   0.8s
[CV 2/5; 1/18] START algorithm=ball_tree, n_neighbors=6, weights=uniform........
[CV 2/5; 1/18] END algorithm=ball_tree, n_neighbors=6, weights=uniform;, score=0.986 total time=   0.7s
[CV 3/5; 1/18] START algorithm=ball_tree, n_neighbors=6, weights=uniform........
[CV 3/5; 1/18] END algorithm=ball_tree, n_neighbors=6, weights=uniform;, score=0.986 total time=   0.4s
[CV 4/5; 1/18] START algorithm=ball_tree, n_neighbors=6, weights=uniform........
[CV 4/5; 1/18] END algorithm=ball_tree, n_neighbors=6, weights=uniform;, score=0.989 total time=   0.4s
[CV 5/5; 1/18] START algorithm=ball_tree, n_neighbors=6, weights=uniform........
[CV 5/5; 1/18] END algorithm=ball_tree, n_neighbors=6, weights=uniform;, score=0.986 total time=   0.4s
[CV 1/5; 2/18]

In [122]:
def KNN_model(X_train, y_train, X_test, y_test, pca):
    X_train = pca.transform(X_train)
    X_test = pca.transform(X_test)
    model = KNeighborsClassifier(algorithm = 'ball_tree', n_neighbors = 6, weights = 'distance')
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    KNN_F1 = f1_score(y_test, y_pred)
    KNN_acc = accuracy_score(y_test, y_pred)
    KNN_cm = confusion_matrix(y_test, y_pred)
    print(f"F1 score: {KNN_F1}")
    print(f"Accuracy: {KNN_acc}")
    print(f"Confussion matrix{KNN_cm}")

In [123]:
KNN_model(X_train, y_train, X_test, y_test,pca)

F1 score: 0.8913963328631875
Accuracy: 0.9486666666666667
Confussion matrix[[2214   36]
 [ 118  632]]



```
With 5:1
F1 score: 0.5
Accuracy: 0.8808808808808809
Confussion matrix[[1641   24]
                  [ 214  119]]

With 3:1
F1 score: 0.8913963328631875
Accuracy: 0.9486666666666667
Confussion matrix[[2214   36]
                  [ 118  632]]
```

### Model#3 LightGBM

In [124]:
def LightGBM_model(X_train, y_train, X_test, y_test):
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, train_size=0.8, random_state = 42)

    train_data = lgb.Dataset(X_train, label=y_train)
    validation_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)

    param = {
             'learning_rate': 0.02,
             'metric': 'binary_logloss',
             'objective': 'binary',
             'min_data_in_leaf': 12
            }

    num_round = 1000
    model = lgb.train(param, train_data, num_round, valid_sets=validation_data, callbacks=[lgb.early_stopping(stopping_rounds=5)])
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)
    y_pred = np.round(y_pred)
    LightGBM_F1 = f1_score(y_test, y_pred)
    LightGBM_acc = accuracy_score(y_test, y_pred)
    LightGBM_cm = confusion_matrix(y_test, y_pred)
    print(f"F1 score: {LightGBM_F1}")
    print(f"Accuracy: {LightGBM_acc}")
    print(f"Confussion matrix{LightGBM_cm}")

In [125]:
LightGBM_model(X_train, y_train, X_test, y_test)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 2992, number of negative: 9008
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002522 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2306
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 22
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.249333 -> initscore=-1.102171
[LightGBM] [Info] Start training from score -1.102171
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 5 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


```
With 5:1
F1 score: 0.9064976228209192
Accuracy: 0.9704704704704705
Confussion matrix[[1653   12]
 [  47  286]]

With 3:1
F1 score: 0.9192634560906515
Accuracy: 0.962
Confussion matrix[[2237   13]
                  [ 101  649]]
```


### Model#4 CatBoosting

In [ ]:
def CatBoosting_GS(X_train, y_train, cat_features):
    model = CatBoostClassifier(loss_function='Logloss', random_state=42, cat_features=cat_features)

    param_grid = {
        'iterations': [250, 300, 350],
        'learning_rate': [0.1, 0.15, 0.2],
        'depth': [9, 11, 13]
    }

    scorer = make_scorer(f1_score, average='weighted')
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring=scorer, n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Print the best hyperparameters for Grid Search
    print("Grid Search - Best Hyperparameters:", grid_search.best_params_)
CatBoosting_GS(X_netrain, y_netrain, cate_features)

In [131]:
def CatBoosting_model(X_train, y_train, X_test, y_test, cat_features):
    model = CatBoostClassifier(iterations=300, depth=9, learning_rate=0.15, cat_features=cat_features,
                           loss_function='Logloss', custom_metric=['AUC'], random_seed=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    CatBoosting_F1 = f1_score(y_test, y_pred)
    CatBoosting_acc = accuracy_score(y_test, y_pred)
    CatBoosting_cm = confusion_matrix(y_test, y_pred)
    print(f"F1 score: {CatBoosting_F1}")
    print(f"Accuracy: {CatBoosting_acc}")
    print(f"Confussion matrix{CatBoosting_cm}")

In [130]:
CatBoosting_model(X_netrain, y_netrain, X_netest, y_netest, cate_features)

0:	learn: 0.5542593	total: 106ms	remaining: 31.7s
1:	learn: 0.4462178	total: 537ms	remaining: 1m 19s
2:	learn: 0.3806321	total: 872ms	remaining: 1m 26s
3:	learn: 0.3282046	total: 1.14s	remaining: 1m 24s
4:	learn: 0.2991043	total: 1.38s	remaining: 1m 21s
5:	learn: 0.2766349	total: 1.63s	remaining: 1m 20s
6:	learn: 0.2513292	total: 1.93s	remaining: 1m 20s
7:	learn: 0.2384531	total: 2.22s	remaining: 1m 20s
8:	learn: 0.2293512	total: 2.53s	remaining: 1m 21s
9:	learn: 0.2190723	total: 2.86s	remaining: 1m 22s
10:	learn: 0.2137148	total: 3.17s	remaining: 1m 23s
11:	learn: 0.2071407	total: 3.56s	remaining: 1m 25s
12:	learn: 0.2035674	total: 4.04s	remaining: 1m 29s
13:	learn: 0.1988396	total: 4.41s	remaining: 1m 30s
14:	learn: 0.1940063	total: 4.78s	remaining: 1m 30s
15:	learn: 0.1894408	total: 5.24s	remaining: 1m 32s
16:	learn: 0.1819854	total: 5.67s	remaining: 1m 34s
17:	learn: 0.1806354	total: 6.03s	remaining: 1m 34s
18:	learn: 0.1777607	total: 6.34s	remaining: 1m 33s
19:	learn: 0.1760288	to



```
With 5:1
F1 score: 0.5907473309608541
Accuracy: 0.8848848848848849
Confussion matrix[[1602   63]
                  [ 167  166]]

With 3:1
F1 score: 0.6536901865369018
Accuracy: 0.8576666666666667
Confussion matrix[[2170   80]
                  [ 347  403]]
```



#TESTING

4-6 dataset: A
7-9 dataset: B

---
with A -> B

We've come to the conclusion that the best ratio is:

##B-> A

##A+B -> Fold

##A+kB => (1-k)B